In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [11]:
URL = "https://ecomideas.com/home-pages-nav"

In [12]:
HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 'Accept-Language' : 'en-US en;q=0.5'})

In [13]:
webpage =requests.get(URL, headers=HEADERS)

In [17]:
webpage

<Response [200]>

In [5]:

def get_image_url(soup):
    try:
        image_tag = soup.find("img", attrs={"alt": "image"})
        image_url = image_tag['src']
    except (AttributeError, TypeError):
        image_url = ""
    return image_url

def get_image_description(soup):
    try:
        description = soup.find("p", attrs={"class": "notion-text notion-text__content notion-semantic-string"}).get_text(separator=" ").strip()
    except AttributeError:
        description = ""
    return description

if __name__ == '__main__':
    HEADERS = ({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36', 
        'Accept-Language': 'en-US, en;q=0.5'
    })

    URL = "https://ecomideas.com/home-pages-nav"
    webpage = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    sub_links = soup.find_all("a", attrs={'class': 'notion-link notion-collection-card__anchor'})
    data = {"image_url": [], "image_description": []}

    for sub_link in sub_links:
        sub_page = requests.get("https://ecomideas.com" + sub_link.get('href'), headers=HEADERS)
        sub_soup = BeautifulSoup(sub_page.content, "html.parser")
        data['image_url'].append(get_image_url(sub_soup))
        data['image_description'].append(get_image_description(sub_soup))

    ecomideas_df = pd.DataFrame.from_dict(data)
    ecomideas_df['image_url'] = ecomideas_df['image_url'].replace('', pd.NA)
    ecomideas_df = ecomideas_df.dropna(subset=['image_url'])

    if not os.path.exists("ecomideas_data"):
        os.makedirs("ecomideas_data")

    filename = "ecomideas_data/all_images_and_descriptions.xlsx"
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        ecomideas_df.to_excel(writer, sheet_name='Sheet1', index=False)
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']
        for idx, url in enumerate(ecomideas_df['image_url']):
            worksheet.write_url(idx + 1, 0, url, string="Image Link")

    print("Data scraping and saving completed successfully.")


Data scraping and saving completed successfully.
